# Transfer Learning on your Own Dataset

You will get to use the power of Transfer learning with **your own dataset**. So in this assignment you are going to retrain the pre-trained VGG16 Model with your own detectable objects. 

![](https://miro.medium.com/max/1000/0*xNjEPIZmPvKeqss6)

For doing so, you should :

* Generate a custom dataset, so take your own images from custom objects (or from the team members).  

* Be sure that you take the images from different angles and different distances, different lighting conditions and different backgrounds for each object. In other words, changing the environment where your objects are while training will lead to a better performance (generalization).

* Use at least 3 diffent opbjects (3 classes) and at most 10 classes where such dataset should have at least 50-100 images per class. (more images should lead to better detection performance) 

* Follow the workflow as [decribed before](https://keras.io/guides/transfer_learning/)

* Show the output of the different (training) steps and the resulting classification on unseen data and answer the related questions in the subsections below

---

### Use the following websites that take you trought all the steps of development.

The first page explains how to use the VGG-16 model with Keras to classify pre-tained images. 
> https://machinelearningmastery.com/use-pre-trained-vgg-model-classify-objects-photographs/





This second page explains the complete proces for classification of images using Keras, they start a problem desciption, data praparation, building a custom CNN model and then optimizing this model. After this they look at the same problem solving it with transfer learning. 

> https://machinelearningmastery.com/how-to-develop-a-convolutional-neural-network-to-classify-photos-of-dogs-and-cats/


You can start at the **"Explore Transfer Learning"** chapter, although you probably need to look back at earlier parts of this page.  

| NOTE: Finetuning by retraining all weights in the network as described in the workflow is Optional |
| --- |


```
Imad Hanzaz, Yannick Urselmann, Jaylong Verschuren
```

# Initialization

load all needed libraries and functions, 
check the previous tutorial how to correctly load keras and other modules

In [ ]:
import tensorflow as tf
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.applications.vgg16 import VGG16
import numpy as np
import matplotlib.pyplot as plt
import os
import pathlib
import PIL
from pathlib import Path
import imghdr
from os import listdir
from numpy import asarray
from numpy import save

print("Current Tensorflow version used is: " + tf.__version__)

: 

# Load dataset & Plot a subset

load your dataset and show a plot of the subset of your data



In [ ]:
train_data_dir = pathlib.Path("./trainDataset")
test_data_dir = pathlib.Path("./testDataset")

image_count_train = len(list(train_data_dir.glob('*/*.png')))
image_count_test = len(list(test_data_dir.glob('*/*.png')))

print('Train set batch size = ' + str(image_count_train))
print('Test set batch size = ' + str(image_count_test))


: 

In [ ]:
truck = list(train_data_dir.glob('truck/*'))
PIL.Image.open(str(truck[3]))

: 

In [ ]:
# Train data
# train_data_dir = "./trainDataset"

image_extensions = [".png", ".jpg"]  # add there all your images file extensions

img_type_accepted_by_tf = ["bmp", "gif", "jpeg", "png"]
for filepath in Path(train_data_dir).rglob("*"):
    if filepath.suffix.lower() in image_extensions:
        img_type = imghdr.what(filepath)
        if img_type is None:
            print(f"{filepath} is not an image")
        elif img_type not in img_type_accepted_by_tf:
            print(f"{filepath} is a {img_type}, not accepted by TensorFlow")

: 

In [ ]:
# Test data
# test_data_dir = "./testDataset"
# image_extensions = [".png", ".jpg"]  # add there all your images file extensions

img_type_accepted_by_tf = ["bmp", "gif", "jpeg", "png"]
for filepath in Path(test_data_dir).rglob("*"):
    if filepath.suffix.lower() in image_extensions:
        img_type = imghdr.what(filepath)
        if img_type is None:
            print(f"{filepath} is not an image")
        elif img_type not in img_type_accepted_by_tf:
            print(f"{filepath} is a {img_type}, not accepted by TensorFlow")

: 

In [ ]:
batch_size = 100
img_height = 224
img_width = 224

: 

In [ ]:
# train data

train_ds = tf.keras.utils.image_dataset_from_directory(
    train_data_dir,
    seed=123,
    image_size=(img_height, img_width),
    batch_size=image_count_train)

: 

In [ ]:
class_names_train = train_ds.class_names
print(class_names_train)

: 

In [ ]:
# test data

test_ds = tf.keras.utils.image_dataset_from_directory(
    test_data_dir,
    seed=123,
    image_size=(img_height, img_width),
    batch_size=image_count_test)

: 

In [ ]:
class_names_test = test_ds.class_names
print(class_names_test)

: 

# Prepare Data

pre-process your raw input data... rescale... normalize....

In [ ]:
for image_batch, labels_batch in train_ds:
    x_train = image_batch
    y_train = labels_batch

y_train_onehot = tf.keras.utils.to_categorical(y_train, 10)

x_train = np.array(x_train)
y_train = np.array(y_train)
y_train_onehot = np.array(y_train_onehot)

: 

In [ ]:
for image_batch, labels_batch in test_ds:
    x_test = image_batch
    y_test = labels_batch

y_test_onehot = tf.keras.utils.to_categorical(y_test, 10)

x_test = np.array(x_test)
y_test = np.array(y_test)
y_test_onehot = np.array(y_test_onehot)

## Define your Model

With Transfer learning you take a given network model withouth the last layers, you can take the suggested VGG-16 model as decribed on the given website and add the additional layers. 

**NOTE:**
That the Ouput layer should match your input dataset!



* How is your model constructed, how many trainable parameters does it have, and where are they located?

In [ ]:
base_model = VGG16(weights="imagenet", include_top=False, input_shape=(224,224,3))   ## Loading VGG16 model
base_model.trainable = False    ## Not trainable weights

print(base_model.summary())

## Preprocessing input
x_train = preprocess_input(x_train) 
x_test = preprocess_input(x_test)

: 

In [ ]:
flatten_layer = tf.keras.layers.Flatten()
dense_layer_1 = tf.keras.layers.Dense(50, activation='relu')
dense_layer_2 = tf.keras.layers.Dense(20, activation='relu')
prediction_layer = tf.keras.layers.Dense(5, activation='softmax')


model = tf.keras.models.Sequential([
    base_model,
    flatten_layer,
    dense_layer_1,
    dense_layer_2,
    prediction_layer
])

: 

# Fit the Model

Fitting the model is the time consuming part, this depend on the complexity of the model and the amount of training data. With Transfer learning a lot of pre-trained parameters are now 'frozen', this will limit training time (or enables us to train more complex networks with the same processing performance, and so achieving better results)

* Which batch size and how many epochs give a good result?

In [ ]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

es = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True)

model.fit(x_train, y_train_onehot, epochs=50, validation_split=0.2, batch_size=32, callbacks=[es])

: 

# Evaluate Model

Show the model accuracy after the training process ... 
* How accurate is your final model?

: 

# learning curves

Show the learning curves of your training sequence, of accuracy, value_accuracy and loss, value_loss



: 

# Save model

Save the model for later usage

: 

# Evaluate Final Model

After training and saving the model you can deploy this model on any given input image. You can start a new application in where you import this model and apply it on any given imput images, so you can just load the model and don't need the timeconsuming training anymore.

: 

# Make Prediction

We can use our saved model to make a prediction on new images that are not trained on... make sure the input images receive the same pre-processing as the images you trained on.

So fetch some images from the internet (similar classes, but not from your dataset), prepare them to fit your network and classify them. Do this for  **10 images per class** and show the results!

* How good is the detection on you real dataset? (show some statistics) 

: 

# Additional Questions
* Explain what your input object represents and how the result should be classified
* Explain the pre-processing steps of the object training image(s) before you can feed it to the network.
* What features do you think are extracted (relevant)?
* Show (in report and video) how accurate your predicted model is, how does your detection behave in other unseen situations? Also explain in what situation and why it does (not) perform well. Supported your statements by measurement data!
* Explain the parameters that you used for re-training this network?
* This example uses a custom (but pre-trained) network architecture, explain how it works and why it is build up this way?

: 

## Finetuning
Finetuning by retraining all weights in the network as described in the workflow is Optional, but this will lead to a better accuracy of your final model,

: 